In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


In [ ]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [ ]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [ ]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

In [ ]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

In [ ]:
%%time
from gensim.models import ldamulticore
lda_model = ldamulticore.LdaMulticore(corpus, num_topics=100, id2word = dictionary, workers = 7, eval_every = None, minimum_probability =0.0)

In [ ]:
# for each query : for every document get topic distribution

In [ ]:
lda_model.save('lda_stem100.model')

In [ ]:
lda_model = gensim.models.LdaModel.load('lda_stem100.model')

In [ ]:
%%time
theta, _ = lda_model.inference(corpus)

In [ ]:
%%time
theta /= theta.sum(axis=1)[:, None]

In [ ]:
#Save
with open('theta.pkl', 'wb') as output:
    pickle.dump(theta, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load 
with open('theta.pkl', 'rb') as ip:
    theta = pickle.load(ip)



In [ ]:
%%time
topics_terms = lda_model.state.get_lambda()
topics_terms = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)


In [ ]:
lda_model.show_topics(-1)

In [ ]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
query_truth['1']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gensim.matutils as gm

In [ ]:
def sim_matrix(query, sim_fn):
    docs = query_truth[str(query)]
    docs.sort(key = lambda x : -int(x[1]))
    n = 0
    for i in docs:
        n = n + int(i[1])
    s  = len(docs)
    m = np.zeros((s,s))
    for i in xrange(s):
        for j in xrange(i + 1, s):
            v1 = theta[filenames.index(docs[i][0] + '.txt')]
            v2 = theta[filenames.index(docs[j][0] + '.txt')]
            if sim_fn == gm.cossim:
                v1 = gm.dense2vec(v1)
                v2 = gm.dense2vec(v2)
            m[i, j] = (1 - sim_fn(v1, v2))
    #print m
    print 'Query', query
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 10) 
    ax.matshow(m, cmap=plt.cm.Blues)
    ax.hlines(n -0.5, n -0.5, s - 0.5, lw=2, color='r')
    ax.vlines(n -0.5, -0.5, n - 0.5, lw=2, color='r')
    plt.show()


In [ ]:
%%time
for query in xrange(1, 11):
    sim_matrix(query, gm.cossim)

In [ ]:
%%time
for query in xrange(1, 11):
    sim_matrix(query, gm.jensen_shannon)

In [ ]:
%%time
for query in xrange(1, 11):
    sim_matrix(query, gm.hellinger)

In [ ]:

import math
from six import iteritems
from six.moves import xrange



class Lda(object):

    def __init__(self, corpus):
        self.corpus_size = dictionary.num_docs
        s = 0
        for i in corpus:
            for x in i:
                s = s + x[1]
        s = float(s)    
        self.avgdl = s / self.corpus_size
        self.corpus = corpus
        self.df = dictionary.dfs
        self.idf = {}
        self.initialize()

    def initialize(self):
        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, query, index, average_idf):
        score = 0
        index_doc = dict(self.corpus[index])
        
        for word,freq in query:
            if word not in index_doc:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += freq*((idf * index_doc[word] * (PARAM_K1 + 1)
                      / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / self.avgdl))))
        return score

    def get_scores(self, query, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, average_idf)
            scores.append(score)
        return scores

